# LLMxBugs

In [1]:
import os
import openai
import pandas as pd
from tqdm import tqdm
import json
import pickle
import numpy as np

In [2]:
openai.api_key = "APIKEY"
MODEL = "text-davinci-002"

In [3]:
def rename_choices(str_desc):
  str_desc = str_desc.replace('1.', '(a)')
  str_desc = str_desc.replace('2.', '(b)')
  str_desc = str_desc.replace('3.', '(c)')
  str_desc = str_desc.replace('4.', '(d)')
  str_desc = '\n'.join([x.lstrip() for x in str_desc.split('\n')])
  return str_desc

## Load Dataset

In [4]:
dataset = pd.read_csv('GameBugDescription.csv')

In [5]:
dataset['Descr1'] = dataset['Descr1'].apply(rename_choices)
dataset['Descr2'] = dataset['Descr2'].apply(rename_choices)

In [6]:
dataset.head()

,File,Game,Descr1,Descr2,Descr1-GT,Descr2-GT,Bug Type,Source
0,ka7yth,Cyberpunk 2077,(a) A person approaches a fence\n(b) A car app...,(a) A car is driving next to a ledge\n(b) The ...,3,3,Collision bug,https://www.reddit.com/r/GamePhysics/comments/...
1,kae98l,Cyberpunk 2077,(a) A car clips through multiple containers\n(...,(a) A car is driving towards a closed shipping...,1,3,Collision bug,https://www.reddit.com/r/GamePhysics/comments/...
2,kbqzb0,Cyberpunk 2077,(a) A car hits another car while driving on th...,(a) The player is driving a car towards anothe...,2,3,Irregular force,https://www.reddit.com/r/GamePhysics/comments/...
3,kbwq31,Cyberpunk 2077,(a) A person pulls out an invisible sword\n(b)...,(a) The player equips a sword in a menu\n(b) T...,1,3,Graphical bug,https://www.reddit.com/r/GamePhysics/comments/...
4,kc7eum,Cyberpunk 2077,(a) A person approaches a policeman\n(b) A hol...,(a) The player walks up to a man\n(b) A game i...,4,4,Sliding objects,https://www.reddit.com/r/GamePhysics/comments/...


In [7]:
q_tempalte = """Q: In the __GAME_NAME__ video game, the following sequence of events happened:

__EVENT__

Which event is a bug?

A:"""

In [8]:
Trigger = "Let's reason the events according to the reference game"

In [13]:
def detect_the_bug(initial_prompt, trigger_sentece=Trigger):
  initial_response = openai.Completion.create(
          model=MODEL,
          prompt=initial_prompt+trigger_sentece,
          temperature=0,
          max_tokens=256,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=['Q:']
  )
  
  answer_extractor_prompt = (
    initial_prompt + trigger_sentece
    + initial_response["choices"][0]["text"]
    + "\n"
    + "Among (a) through (d), the answer is"
  )
  
  response_final = openai.Completion.create(
    model=MODEL,
    prompt=answer_extractor_prompt,
    temperature=0,
    max_tokens=32,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
  )
  
  return initial_prompt+trigger_sentece, initial_response, answer_extractor_prompt, response_final

In [24]:
def run_single_query(query_id, descr='Descr1', q_tempalte=q_tempalte, trigger_sentece=Trigger):
  event_desc = dataset[descr][query_id]
  game       = dataset['Game'][query_id]

  prompt_q_init = q_tempalte.replace('__EVENT__', event_desc).replace('__GAME_NAME__', game)
  # print(prompt_q_init+trigger_sentece)
  return detect_the_bug(prompt_q_init)

In [25]:
input_a, output_a, input_b, output_b = run_single_query(79, descr='Descr1')

In [26]:
print(input_b)

Q: In the Grand Theft Auto V video game, the following sequence of events happened:

(a) A person is driving a car on a freeway
(b) The person jumps out of the vehicle
(c) The person is hit by another car and ejected into the sky
(d) The person falls from the sky and hits the beach

Which event is a bug?

A:Let's reason the events according to the reference game's mechanics. In GTA V, when a player exits a vehicle while it's still in motion, the character will automatically enter a "ragdoll" state and fall to the ground. If the player is then hit by another vehicle while in this ragdoll state, they will be ejected into the air. However, the character should then fall back down to the ground and not remain floating in the air. Therefore, event (c) is the bug.
Among (a) through (d), the answer is


In [27]:
output_b["choices"][0]["text"]

' (c).'